#### Parametros para todo el notebook

In [1]:
# Candlesticks
bar_size = 1000000

#Labels
horizons=[50, 100]
betas=[1.5, 2]
periods=[14]
direction='long'

# Model training
epochs=500
batch_size=128
patience=50


In [2]:
from lib import *

import os
import importlib
import tensorflow as tf


#### Levanto datos y armo velas de volumen

In [3]:
#from lib import DataHandler, FeaturesGenerator, Labeler
# Inicializo
dh = DataHandler("./data.csv", 
                 tick_data=False)
# Genero diferentes data con volume bars
vol_bars = dh.generate_volumebars(bar_size)



#### Genero los features

In [4]:
# Genero features para cada data por separado
fg = FeaturesGenerator(vol_bars, 
                         momentum=True, 
                         volume=False, 
                         volatility=False)
features = fg.df




#### Genero los labels

In [5]:
lb = Labeler(vol_bars,
            horizons=horizons,
            betas=betas,
            periods=periods,
            direction=direction)




In [6]:
labels = lb.df

#### Chequeo balance entre etiquetas

In [7]:
for column in labels.columns:
    print(labels[column].value_counts(dropna=False))

h_50, b_1.5, p_14, ATR
1.0    723
0.0    502
NaN     50
Name: count, dtype: int64
h_50, b_1.5, p_14, V
1.0    656
0.0    569
NaN     50
Name: count, dtype: int64
h_50, b_2, p_14, ATR
1.0    757
0.0    468
NaN     50
Name: count, dtype: int64
h_50, b_2, p_14, V
1.0    693
0.0    532
NaN     50
Name: count, dtype: int64
h_100, b_1.5, p_14, ATR
1.0    683
0.0    492
NaN    100
Name: count, dtype: int64
h_100, b_1.5, p_14, V
1.0    622
0.0    553
NaN    100
Name: count, dtype: int64
h_100, b_2, p_14, ATR
1.0    708
0.0    467
NaN    100
Name: count, dtype: int64
h_100, b_2, p_14, V
1.0    656
0.0    519
NaN    100
Name: count, dtype: int64


#### Pre procesamos: Limpieza de nan, train-test split y normalizacion de features

In [8]:
dpp = DataPreprocessor(vol_bars, features, labels, validation_size=0.2, test_size=0.3)
X_train, X_val, X_test, y_train, y_val, y_test = dpp.preprocess()

In [9]:
input_shape = X_train.shape[1:]
input_shape

(65,)

In [11]:
model_config_files = [f[:-3] for f in os.listdir('model_configs') if f.endswith('.py') and f != '__init__.py']
all_metrics = []

for config_file in model_config_files:
    i=0
    config_module = importlib.import_module(f'model_configs.{config_file}')
    model_name = config_module.CONFIG['model_name']
    model_type = config_module.CONFIG['model_type']
    model = config_module.build_model

    for i in range(y_train.shape[1]):
        label_option = y_train.columns[i]

        mh = ModelsHandler(model_name=model_name, 
                           model_type=model_type,
                           direction=direction,
                           label_option=label_option)
        
        mh.train_model(build_model_func=model,
                    X_train=X_train,
                    y_train=y_train.iloc[:,i],
                    X_val=X_val,
                    y_val=y_val.iloc[:,i],
                    epochs=epochs,
                    batch_size=batch_size,
                    patience=patience,
                    input_shape=input_shape
                    )

        mh.plot_learning_curves()

        # Identify the epoch with the best validation loss
        best_epoch_index = np.argmax(mh.history.history['val_accuracy'])
        best_metrics = {metric: values[best_epoch_index] for metric, values in mh.history.history.items()}
        
        # Add model name and type to the best metrics
        best_metrics['model_name'] = model_name
        best_metrics['model_type'] = model_type
        best_metrics['label_option'] = label_option

        all_metrics.append(best_metrics)

        i+=1

# Convert the list of best metrics dictionaries to a DataFrame
metrics_df = pd.DataFrame(all_metrics)
# Sort the DataFrame by validation accuracy
metrics_df.sort_values(by='val_accuracy', ascending=False, inplace=True)
metrics_df.to_csv('metrics.csv')


    


    




Epoch 1/500
1/4 [======>.......................] - ETA: 5s - loss: 0.7367 - accuracy: 0.5234
Epoch 1: val_accuracy improved from -inf to 0.38039, saving model to model_weights\model1_h_50, b_1.5, p_14, ATR_best_weights.h5
4/4 [==============================] - 2s 130ms/step - loss: 0.7934 - accuracy: 0.5350 - val_loss: 0.7837 - val_accuracy: 0.3804
Epoch 2/500
1/4 [======>.......................] - ETA: 0s - loss: 0.8353 - accuracy: 0.4531
Epoch 2: val_accuracy improved from 0.38039 to 0.65490, saving model to model_weights\model1_h_50, b_1.5, p_14, ATR_best_weights.h5
4/4 [==============================] - 0s 41ms/step - loss: 0.7352 - accuracy: 0.5412 - val_loss: 0.6330 - val_accuracy: 0.6549


c:\Users\maxgr\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch 3/500
1/4 [======>.......................] - ETA: 0s - loss: 0.8040 - accuracy: 0.5391
Epoch 3: val_accuracy did not improve from 0.65490
4/4 [==============================] - 0s 34ms/step - loss: 0.7608 - accuracy: 0.5658 - val_loss: 0.6638 - val_accuracy: 0.5882
Epoch 4/500
1/4 [======>.......................] - ETA: 0s - loss: 0.7159 - accuracy: 0.5625
Epoch 4: val_accuracy did not improve from 0.65490
4/4 [==============================] - 0s 27ms/step - loss: 0.7139 - accuracy: 0.5412 - val_loss: 0.6970 - val_accuracy: 0.5059
Epoch 5/500
1/4 [======>.......................] - ETA: 0s - loss: 0.6941 - accuracy: 0.5781
Epoch 5: val_accuracy did not improve from 0.65490
4/4 [==============================] - 0s 26ms/step - loss: 0.7021 - accuracy: 0.5638 - val_loss: 0.6397 - val_accuracy: 0.6549
Epoch 6/500
1/4 [======>.......................] - ETA: 0s - loss: 0.7319 - accuracy: 0.5625
Epoch 6: val_accuracy did not improve from 0.65490
4/4 [==============================] - 0

c:\Users\maxgr\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch 3/500
1/4 [======>.......................] - ETA: 0s - loss: 0.7056 - accuracy: 0.5781
Epoch 3: val_accuracy improved from 0.55686 to 0.56078, saving model to model_weights\model1_h_50, b_1.5, p_14, V_best_weights.h5
4/4 [==============================] - 0s 47ms/step - loss: 0.7122 - accuracy: 0.5658 - val_loss: 0.6775 - val_accuracy: 0.5608
Epoch 4/500
1/4 [======>.......................] - ETA: 0s - loss: 0.6960 - accuracy: 0.5234
Epoch 4: val_accuracy did not improve from 0.56078
4/4 [==============================] - 0s 28ms/step - loss: 0.6880 - accuracy: 0.5494 - val_loss: 0.6844 - val_accuracy: 0.5294
Epoch 5/500
1/4 [======>.......................] - ETA: 0s - loss: 0.6829 - accuracy: 0.5781
Epoch 5: val_accuracy improved from 0.56078 to 0.61176, saving model to model_weights\model1_h_50, b_1.5, p_14, V_best_weights.h5
4/4 [==============================] - 0s 44ms/step - loss: 0.6797 - accuracy: 0.5556 - val_loss: 0.6623 - val_accuracy: 0.6118
Epoch 6/500
1/4 [======>..

...


Epoch 2: val_accuracy improved from 0.44314 to 0.55686, saving model to model_weights\model2_h_100, b_1.5, p_14, V_best_weights.h5
4/4 [==============================] - 0s 68ms/step - loss: 0.8229 - accuracy: 0.4774 - val_loss: 0.7110 - val_accuracy: 0.5569
Epoch 3/500
4/4 [==============================] - ETA: 0s - loss: 0.7847 - accuracy: 0.5453
Epoch 3: val_accuracy did not improve from 0.55686
4/4 [==============================] - 0s 88ms/step - loss: 0.7847 - accuracy: 0.5453 - val_loss: 0.6757 - val_accuracy: 0.5569
Epoch 4/500
4/4 [==============================] - ETA: 0s - loss: 0.6681 - accuracy: 0.6091
Epoch 4: val_accuracy did not improve from 0.55686
4/4 [==============================] - 0s 69ms/step - loss: 0.6681 - accuracy: 0.6091 - val_loss: 0.6984 - val_accuracy: 0.5255
Epoch 5/500
4/4 [==============================] - ETA: 0s - loss: 0.7153 - accuracy: 0.4938
Epoch 5: val_accuracy did not improve from 0.55686
4/4 [==============================] - 0s 70ms/step 

c:\Users\maxgr\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


4/4 [==============================] - 0s 47ms/step - loss: 0.7737 - accuracy: 0.5391 - val_loss: 0.6611 - val_accuracy: 0.7020
Epoch 3/500
1/4 [======>.......................] - ETA: 0s - loss: 0.6931 - accuracy: 0.6719
Epoch 3: val_accuracy did not improve from 0.70196
4/4 [==============================] - 0s 36ms/step - loss: 0.7312 - accuracy: 0.6523 - val_loss: 0.6042 - val_accuracy: 0.7020
Epoch 4/500
1/4 [======>.......................] - ETA: 0s - loss: 0.7786 - accuracy: 0.5312
Epoch 4: val_accuracy did not improve from 0.70196
4/4 [==============================] - 0s 35ms/step - loss: 0.7430 - accuracy: 0.5062 - val_loss: 0.6595 - val_accuracy: 0.6157
Epoch 5/500
1/4 [======>.......................] - ETA: 0s - loss: 0.7006 - accuracy: 0.4844
Epoch 5: val_accuracy did not improve from 0.70196
4/4 [==============================] - 0s 94ms/step - loss: 0.6659 - accuracy: 0.5782 - val_loss: 0.5954 - val_accuracy: 0.7020
Epoch 6/500
2/4 [==============>...............] - ETA: 

c:\Users\maxgr\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


4/4 [==============================] - 0s 49ms/step - loss: 0.7881 - accuracy: 0.5062 - val_loss: 0.6726 - val_accuracy: 0.6000
Epoch 3/500
1/4 [======>.......................] - ETA: 0s - loss: 0.7505 - accuracy: 0.4844
Epoch 3: val_accuracy did not improve from 0.60000
4/4 [==============================] - 0s 34ms/step - loss: 0.7410 - accuracy: 0.5247 - val_loss: 0.6743 - val_accuracy: 0.5961
Epoch 4/500
4/4 [==============================] - ETA: 0s - loss: 0.7431 - accuracy: 0.5123
Epoch 4: val_accuracy did not improve from 0.60000
4/4 [==============================] - 0s 42ms/step - loss: 0.7431 - accuracy: 0.5123 - val_loss: 0.6924 - val_accuracy: 0.4824
Epoch 5/500
1/4 [======>.......................] - ETA: 0s - loss: 0.7625 - accuracy: 0.4453
Epoch 5: val_accuracy did not improve from 0.60000
4/4 [==============================] - 0s 40ms/step - loss: 0.7039 - accuracy: 0.5412 - val_loss: 0.6752 - val_accuracy: 0.5922
Epoch 6/500
1/4 [======>.......................] - ETA: 

In [ ]:
# proximos pasos

# agregar timesteps para LSTM
# crear metrica custom para que sea robusta a desbalance de clases
